# Análise Comparativa de Modelos de Machine Learning para o Conjunto de Dados de Diamantes
- **Tratamento de dados faltantes**: Identificar e remover linhas/colunas com dados faltantes insignificantes. Imputar valores ausentes usando estatísticas de tendência central ou modelos preditivos;
- **Tratamento de dados discrepantes**: remover ou isolar outliers dependendo do contexto;
- **Codificação de variáveis**: efetuar toda e qualquer transformação necessária para o funcionamento do modelo, seja ela codificação de variáveis qualitativas (nominais ou ordinais), temporais ou textuais;
- **Validação cruzada**: Deve-se utilizar um método de validação cruzada (holdout, k-fold, Monte Carlo) apresentado em sala de aula;

### Importando Bibliotecas

In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, cross_validate

### Leitura dos dados

In [42]:
df= pd.read_csv("../data/raw/data.csv")
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### Dicionário

In [14]:
df_dict = pd.read_csv('../data/external/dictionary.csv')
df_dict

,variavel,descricao,tipo,subtipo
0,price,"price in US dollars ($326--$18,823)",quantitativa,contínua
1,carat,weight of the diamond (0.2--5.01),quantitativa,contínua
2,cut,"quality of the cut (Fair, Good, Very Good, Pre...",qualitativa,ordinal
3,color,"diamond colour, from D (best) to J (worst)",qualitativa,ordinal
4,clarity,a measurement of how clear the diamond is (I1 ...,qualitativa,ordinal
5,x,length in mm (0--10.74),quantitativa,contínua
6,y,width in mm (0--58.9),quantitativa,contínua
7,z,depth in mm (0--31.8),quantitativa,contínua
8,depth,"total depth percentage = z / mean(x, y) = 2 * ...",quantitativa,contínua
9,table,width of top of diamond relative to widest poi...,quantitativa,contínua


### Renomeando as colunas

In [68]:
mapeamento_colunas = {
    'price': 'preço',  
    'carat': 'peso do diamante',
    'cut': 'qualidade do corte',
    'color': 'cor do diamante',
    'clarity': 'clareza do diamante',
    'x': 'comprimento em mm',
    'y': 'largura em mm',
    'z': 'profundidade em mm',
    'depth': 'porcentagem de profundidade',
    'table': 'largura superior do diamante'
}

In [71]:
def renomear_colunas(df, mapping):
    df.rename(columns=mapping, inplace=True)
    return df

In [70]:
df_dict['variavel'] = df_dict['variavel'].replace(mapeamento_colunas)
df = renomear_colunas(df, mapeamento_colunas)

## 1. Tratamento e Transformação de Dados

### Verificando os Valores Faltantes e Dados Duplicados

In [19]:
print(df.duplicated().sum())

0


In [18]:
df = df.drop(df[df.duplicated()].index, axis=0)
df.reset_index()
print(f'Número de Duplicados: {df.duplicated().sum()}')

Número de Duplicados: 0


In [20]:
print(f'Verificando valores faltantes : \n\n{df.isna().sum()}')

Verificando valores faltantes : 

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64


### Verificando os Valores Discrepantes

In [21]:
# Função para identificar outliers usando o IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # Primeiro quartil
    Q3 = df[column].quantile(0.75)  # Terceiro quartil
    IQR = Q3 - Q1                     # Intervalo Interquartil
    
    lower_bound = Q1 - 1.5 * IQR      # Limite inferior
    upper_bound = Q3 + 1.5 * IQR      # Limite superior
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

In [22]:
# Detectando outliers apenas para colunas numéricas
outliers_summary = []

for col in df.select_dtypes(include=[np.number]).columns:  # Filtra apenas colunas numéricas
    outliers = detect_outliers_iqr(df, col)
    if not outliers.empty:
        outliers_summary.append({'Coluna': col, 'Quantidade de Outliers': len(outliers)})

outliers_df = pd.DataFrame(outliers_summary)

print(outliers_df)

  Coluna  Quantidade de Outliers
0  carat                    1873
1  depth                    2525
2  table                     604
3  price                    3523
4      x                      31
5      y                      28
6      z                      48


In [25]:
df['price'] = np.log1p(df['price'])

## 2. Pré-Processamento dos Dados

### Definição das Variáveis do Modelo de Previsão

In [43]:
# Variável alvo
target_variable = 'price'

# Variáveis inúteis
useless_variables = (
    df_dict
    .query("tipo == 'inútil'")  # As variáveis inúteis devem ser definidas manualmente
    .variavel
    .to_list()
)

# Variáveis ordinais (cut, color, clarity)
ordinal_variables = (
    df_dict
    .query("subtipo == 'ordinal' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis contínuas (carat, x, y, z, depth, table)
continuous_variables = (
    df_dict
    .query("subtipo == 'contínua' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis nominais (nenhuma claramente nominal com base nas definições)
nominal_variables = []  # Como 'cut', 'color' e 'clarity' são ordinais, não serão categorizadas aqui

# Variáveis discretas (não presentes no dataset, então a lista será vazia)
discrete_variables = []  # Não há variáveis discretas definidas para esse dataset

# Exibição das variáveis categorizadas
print("Variáveis Ordinais:", ordinal_variables)
print("Variáveis Contínuas:", continuous_variables)
print("Variáveis Nominais:", nominal_variables)
print("Variáveis Discretas:", discrete_variables)


Variáveis Ordinais: ['cut', 'color', 'clarity']
Variáveis Contínuas: ['carat', 'x', 'y', 'z', 'depth', 'table']
Variáveis Nominais: []
Variáveis Discretas: []


In [27]:
X = df.drop(columns=[target_variable] + useless_variables)
y = df[target_variable]

In [28]:
# Exibindo as categorias de variáveis
print("Nominal Variables:", nominal_variables)
print("Ordinal Variables:", ordinal_variables)
print("Continuous Variables:", continuous_variables)
print("Discrete Variables:", discrete_variables)

Nominal Variables: []
Ordinal Variables: ['cut', 'color', 'clarity']
Continuous Variables: ['carat', 'x', 'y', 'z', 'depth', 'table']
Discrete Variables: []


### Pré-processamento das Variáveis: Imputação e Codificação

In [60]:
# Ordinal variables: imputação de valores faltantes e codificação ordinal
ordinal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="most_frequent")),  # Preenchimento com a moda (mais frequente)
    ("encoding", OrdinalEncoder())  # Codificação ordinal
])

# Continuous variables: imputação de valores faltantes e normalização
continuous_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="mean")),  # Preenchimento com a média
    ("normalization", StandardScaler())  # Normalização
])

In [61]:
# Criação do ColumnTransformer para processar cada tipo de variável
preprocessor = ColumnTransformer([  
    ("ordinal", ordinal_preprocessor, ordinal_variables),   
    ("continuous", continuous_preprocessor, continuous_variables) 
])

In [62]:
preprocessor

ColumnTransformer(transformers=[('ordinal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OrdinalEncoder())]),
                                 ['cut', 'color', 'clarity']),
                                ('continuous',
                                 Pipeline(steps=[('missing', SimpleImputer()),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['carat', 'x', 'y', 'z', 'depth', 'table'])])